<a href="https://colab.research.google.com/github/Iremguel/Fallstudie_Elektrifizierung_der_Logistik/blob/main/Stufe_3_(%2B_Batteriedynamik_%26_Laden).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Fallstudie Elektrifizierung

In [15]:
! pip install -q pyscipopt

In [16]:
import pandas as pd
from pyscipopt import Model, quicksum

In [17]:
! git clone https://github.com/Iremguel/Fallstudie_Elektrifizierung_der_Logistik.git

fatal: destination path 'Fallstudie_Elektrifizierung_der_Logistik' already exists and is not an empty directory.


In [18]:
! ls

Fallstudie_Elektrifizierung_der_Logistik  sample_data


In [19]:
! ls Daten

ls: cannot access 'Daten': No such file or directory


In [20]:
folder = "Fallstudie_Elektrifizierung_der_Logistik/Daten/"

In [21]:
chargers = pd.read_csv(f"{folder}chargers.csv", sep=";")

In [22]:
diesel_trucks = pd.read_csv(f"{folder}/diesel_trucks.csv", sep=";")

In [23]:
electric_trucks = pd.read_csv(f"{folder}/electric_trucks.csv", sep=";")

In [24]:
routes = pd.read_csv(f"{folder}/routes.csv", sep=";")

## Hilfunktionen

In [25]:
def time_to_slot(t):
    """Wandelt HH:MM in 15-Minuten-Zeitschritte um"""
    h, m = map(int, t.split(":"))
    return (60 * h + m) // 15

routes["start"] = routes["starttime"].apply(time_to_slot)
routes["end"]   = routes["endtime"].apply(time_to_slot)

## Indexmengen

In [26]:
R = list(routes["route_id"].unique()) # Routen

#LKWs
#K_D = list(diesel_trucks["truck_model"].unique()) # Diesel LKW-Typen
K_E = list(electric_trucks["truck_model"].unique()) # e-LKW Typen
#K = K_D + K_E # Gesamte LKW-Flotte

V = list(range(len(R)))      # physische LKWs (Ausprägung)

S = list(chargers["charger_model"].unique()) # Ladesäulentypen
P = [0, 1] # Ladepunkte je Ladesäule (maximal 2)

T = list(range(96))  # 24h * 4
Δt = 0.25            # Stunden pro Zeitschritt

In [27]:
DAYS = list(range(260))

## Parameter

In [28]:
# Diesel-LKW-Parameter (nur ein Typ)
d = "ActrosL"

diesel_capex = diesel_trucks.set_index("truck_model").loc[d, "capex_yearly"]
diesel_opex  = diesel_trucks.set_index("truck_model").loc[d, "opex_yearly"]
diesel_kfz   = diesel_trucks.set_index("truck_model").loc[d, "kfz_yearly"]
diesel_cons  = diesel_trucks.set_index("truck_model").loc[d, "avg_diesel_per_100km"]

# Konstanten
diesel_price = 1.60  # €/Liter
toll_price   = 0.34  # €/km
DAYS = 260           # Betriebstage pro Jahr

# Fixkosten pro LKW (jährlich)
diesel_fixed_cost = diesel_capex + diesel_opex + diesel_kfz

In [29]:
# Tourenparameter
route_start = routes.set_index("route_id")["start"].to_dict()
route_end   = routes.set_index("route_id")["end"].to_dict()
distance_total = routes.set_index("route_id")["distance_total"].to_dict()
distance_toll  = routes.set_index("route_id")["distance_toll"].to_dict()

In [30]:
# e-LKWs
# Kosten und Erlöse
electric_capex = electric_trucks.set_index("truck_model")["capex_yearly"].to_dict()
electric_opex  = electric_trucks.set_index("truck_model")["opex_yearly"].to_dict()
thg_revenue    = electric_trucks.set_index("truck_model")["thg_yearly"].to_dict()

# Energie und Technik
electric_consumption = electric_trucks.set_index("truck_model")["avg_energy_kWh_per_100km"].to_dict()
truck_max_power      = electric_trucks.set_index("truck_model")["max_power"].to_dict()
soc_max              = electric_trucks.set_index("truck_model")["soc_max_kWh"].to_dict()
electricity_price = 0.25  # €/kWh

In [31]:
# Routenkosten pro Jahr
diesel_route_cost = {}
for r in R:
    fuel = diesel_cons / 100 * distance_total[r] * diesel_price
    toll = distance_toll[r] * toll_price
    diesel_route_cost[r] = DAYS * (fuel + toll)

electric_route_cost = {}
for r in R:
    for k in K_E:
        energy_cost = (
            electric_consumption[k] / 100
            * distance_total[r]
            * electricity_price
        )
        electric_route_cost[r, k] = DAYS * energy_cost

In [32]:
#Ladesäulen
charger_capex = chargers.set_index("charger_model")["capex_yearly"].to_dict()
charger_opex  = chargers.set_index("charger_model")["opex_yearly"].to_dict()
charger_power = chargers.set_index("charger_model")["max_power"].to_dict()
charger_spots = chargers.set_index("charger_model")["charging_spots"].to_dict()

In [33]:
# Batteriespeicher-Parameter
batt_capex_power = 30    # €/kW/year
batt_capex_energy = 350  # €/kWh/year
batt_opex_rate = 0.02
battery_efficiency = 0.98
battery_dod = 0.975

## Modellinstanz ziehen

In [34]:
model = Model()

## Entscheidungsvariablen

In [35]:
# Tour–LKW-Zuordnung
# x[r,v] = 1 wenn Route r von LKW v gefahren wird
X_D = {}
for r in R:
    for v in V:
        X_D[r, v] = model.addVar(vtype="B", name=f"xD_{r}_{v}")

X_E = {}
for r in R:
    for v in V:
      for k in K_E:
        X_E[r, v, k] = model.addVar(vtype="B", name=f"xE_{r}_{v}_{k}")

In [36]:
# y[v] = 1 wenn LKW v genutzt wird
Y_D = {}
for v in V:
    Y_D[v] = model.addVar(vtype="B", name=f"yD_{v}")

Y_E = {}
for v in V:
  for k in K_E:
    Y_E[v, k] = model.addVar(vtype="B", name=f"yE_{v}_{k}")

In [37]:
# soc[v, t] = Batteriestand von LKW v zum Zeitpunkt t (kWh)
SOC = {(v, t): model.addVar(lb=0, vtype="C", name=f"soc_{v}_{t}")
       for v in V for t in T}

In [38]:
# p[v, t] = Ladeleistung von LKW v in Zeitschritt t (kW)
P_charge = {(v, t): model.addVar(lb=0, vtype="C", name=f"p_charge_{v}_{t}")
            for v in V for t in T}

In [39]:
# z[v, s, p, t] = 1 wenn LKW v an Ladesäule s, Ladepunkt p, Zeit t lädt
Z = {(v, s, p, t): model.addVar(vtype="B", name=f"z_{v}_{s}_{p}_{t}")
     for v in V for s in S for p in P for t in T}

In [40]:
# n[s] = Anzahl installierter Ladesäulen vom Typ s
N_charger = {s: model.addVar(vtype="I", lb=0, ub=3, name=f"n_charger_{s}")
             for s in S}

## Zielfunktion

In [41]:
model.setObjective(
    # Diesel Fixkosten
    quicksum(diesel_fixed_cost * Y_D[v] for v in V)
    +
    # e-LKW Fixkosten minus THG
    quicksum(
        (electric_capex[k] + electric_opex[k] - thg_revenue[k]) * Y_E[v, k]
        for v in V for k in K_E
    )
    +
    # Variable Kosten
    quicksum(diesel_route_cost[r] * X_D[r, v] for r in R for v in V)
    +
    quicksum
        (electric_route_cost[r, k] * X_E[r, v, k]
        for r in R for v in V for k in K_E
    )
    +
    quicksum(
        (charger_capex[s] + charger_opex[s]) * N_charger[s]
        for s in S
    )
    ,
    sense="minimize"

)

## Nebenbedingungen

In [42]:
# Jede Route wird genau einmal gefahren (Diesel ODER e-LKW)
for r in R:
    model.addCons(
        quicksum(X_D[r, v] for v in V)
        + quicksum(X_E[r, v, k] for v in V for k in K_E)
        == 1,
        name=f"route_coverage_{r}"
    )


In [43]:
# Route nur wenn LKW existiert
for r in R:
    for v in V:
        model.addCons(X_D[r, v] <= Y_D[v])
        for k in K_E:
            model.addCons(X_E[r, v, k] <= Y_E[v, k])

In [44]:
# Ein physischer LKW ist ENTWEDER Diesel ODER ein e-LKW-Typ
for v in V:
    model.addCons(
        Y_D[v] + quicksum(Y_E[v, k] for k in K_E) <= 1,
        name=f"one_vehicle_type_{v}"
    )

In [45]:
# Keine zeitliche Überlappung auf demselben LKW
for v in V:
    for i, r1 in enumerate(R):
        for r2 in R[i+1:]:
            if not (
                route_end[r1] <= route_start[r2]
                or route_end[r2] <= route_start[r1]
            ):
                model.addCons(
                    X_D[r1, v] + X_D[r2, v]
                    + quicksum(X_E[r1, v, k] + X_E[r2, v, k] for k in K_E)
                    <= 1,
                    name=f"overlap_{r1}_{r2}_{v}"
                )

In [46]:
model.addCons(quicksum(N_charger[s] for s in S) <= 3)

c3621

In [47]:
# SOC-Grenzen je e-LKW-Typ
for v in V:
    for k in K_E:
        for t in T:
            model.addCons(
                SOC[v, t] <= soc_max[k] * Y_E[v, k]
            )

In [48]:
# SOC-Dynamik
for v in V:
    for t in T[:-1]:
        model.addCons(
            SOC[v, t+1] == SOC[v, t] + P_charge[v, t] * Δt
        )

In [49]:
# Zyklischer Tageszustand
for v in V:
    model.addCons(
        SOC[v, 0] == SOC[v, T[-1]]
    )

In [50]:
M = 1e4  # Big-M

for v in V:
    for t in T:
        model.addCons(
            P_charge[v, t]
            <= quicksum(
                charger_power[s] * Z[v, s, p, t]
                for s in S for p in P
            )
        )

In [51]:
# Ein Ladepunkt – max. ein LKW
for s in S:
    for p in P:
        for t in T:
            model.addCons(
                quicksum(Z[v, s, p, t] for v in V)
                <= N_charger[s]
            )

In [52]:
# Ein LKW lädt max. an einem Ladepunkt gleichzeitig
for v in V:
    for t in T:
        model.addCons(
            quicksum(Z[v, s, p, t] for s in S for p in P)
            <= 1
        )

## Lösung

In [53]:
#model.redirectOutput()

In [54]:
model.optimize()

In [55]:
print("\n==============================")
print("OPTIMALE LÖSUNG (OHNE LADEN)")
print("==============================")
print(f"Gesamtkosten pro Jahr: {model.getObjVal():,.2f} €")

print("\nEingesetzte Fahrzeuge:")
for v in V:
    if model.getVal(Y_D[v]) > 0.5:
        print(f"LKW {v}: Diesel")
    for k in K_E:
        if model.getVal(Y_E[v, k]) > 0.5:
            print(f"LKW {v}: e-LKW {k}")

print("\nRoutenplanung:")
for r in R:
    for v in V:
        if model.getVal(X_D[r, v]) > 0.5:
            print(f"Route {r} → Diesel LKW {v}")
        for k in K_E:
            if model.getVal(X_E[r, v, k]) > 0.5:
                print(f"Route {r} → e-LKW {k} (LKW {v})")


OPTIMALE LÖSUNG (OHNE LADEN)
Gesamtkosten pro Jahr: 851,418.35 €

Eingesetzte Fahrzeuge:
LKW 0: e-LKW eActros400
LKW 1: e-LKW eActros400
LKW 2: Diesel
LKW 3: e-LKW eActros400
LKW 4: Diesel
LKW 6: e-LKW eActros400
LKW 10: e-LKW eActros400
LKW 11: Diesel
LKW 12: Diesel
LKW 13: Diesel
LKW 15: Diesel
LKW 17: Diesel
LKW 18: e-LKW eActros400
LKW 19: Diesel

Routenplanung:
Route t-4 → e-LKW eActros400 (LKW 3)
Route t-5 → e-LKW eActros400 (LKW 0)
Route t-6 → e-LKW eActros400 (LKW 18)
Route s-1 → e-LKW eActros400 (LKW 6)
Route s-2 → e-LKW eActros400 (LKW 10)
Route s-3 → Diesel LKW 15
Route s-4 → e-LKW eActros400 (LKW 1)
Route w1 → Diesel LKW 4
Route w2 → Diesel LKW 13
Route w3 → Diesel LKW 19
Route w4 → Diesel LKW 2
Route w5 → Diesel LKW 17
Route w6 → Diesel LKW 11
Route w7 → Diesel LKW 12
Route r1 → e-LKW eActros400 (LKW 1)
Route r2 → e-LKW eActros400 (LKW 18)
Route r3 → e-LKW eActros400 (LKW 10)
Route h3 → e-LKW eActros400 (LKW 0)
Route h4 → e-LKW eActros400 (LKW 3)
Route k1 → e-LKW eActros4